In [2]:
import json
import importlib
from nba_api.stats import endpoints


# Sample parameters for testing
SAMPLE_PARAMS = {
    'game_id': '0022300001',
    'player_id': '2544',  # LeBron James
    'team_id': '1610612747',  # Lakers
    'season': '2023-24',
    'season_type': 'Regular Season'
}

def get_unknown_endpoints():
    """Load all endpoints marked as 'unknown' priority from the config file"""
    with open('endpoint_priority_review.json', 'r') as f:
        config = json.load(f)
    
    unknown_endpoints = []
    for endpoint_name, endpoint_info in config['endpoints'].items():
        if endpoint_info['priority'] == 'unknown':
            unknown_endpoints.append({
                'name': endpoint_name,
                'required_params': endpoint_info['required_params']
            })
    
    return unknown_endpoints

def setup_endpoint_for_testing(endpoint_name, required_params):
    """
    Setup an endpoint for testing by creating the import statement and 
    parameter mapping needed for manual testing
    """
    # Build the params dict using only required params
    test_params = {}
    for param in required_params:
        if param in SAMPLE_PARAMS:
            test_params[param] = SAMPLE_PARAMS[param]
        else:
            test_params[param] = f"NEED_VALUE_FOR_{param.upper()}"
    
    # Create the code template
    code_template = f"""
# Testing {endpoint_name}
from nba_api.stats.endpoints import {endpoint_name}

# Create instance with parameters
{endpoint_name.lower()}_instance = {endpoint_name}({', '.join([f"{k}='{v}'" for k, v in test_params.items()])})

# Get dataframes (try index 0 first, then 1, 2, etc. if needed)
try:
    df = {endpoint_name.lower()}_instance.get_data_frames()[0]
    print(f"Shape: {{df.shape}}")
    print(f"Columns: {{list(df.columns)}}")
    print("First few rows:")
    display(df.head())
except Exception as e:
    print(f"Error with index 0: {{e}}")
    print("Available datasets:", len({endpoint_name.lower()}_instance.get_data_frames()))
"""
    
    return {
        'endpoint_name': endpoint_name,
        'required_params': required_params,
        'test_params': test_params,
        'code_template': code_template
    }

# Load unknown endpoints
unknown_endpoints = get_unknown_endpoints()
print(f"Found {len(unknown_endpoints)} unknown endpoints:")
for i, endpoint in enumerate(unknown_endpoints[:10]):  # Show first 10
    print(f"{i+1}. {endpoint['name']} - requires: {endpoint['required_params']}")

if len(unknown_endpoints) > 10:
    print(f"... and {len(unknown_endpoints) - 10} more")

Found 64 unknown endpoints:
1. GameRotation - requires: ['game_id']
2. HustleStatsBoxScore - requires: ['game_id']
3. PlayerGameStreakFinder - requires: []
4. PlayerNextNGames - requires: ['player_id']
5. TeamGameLog - requires: ['team_id']
6. TeamGameLogs - requires: []
7. TeamGameStreakFinder - requires: []
8. CumeStatsPlayer - requires: ['player_id', 'game_ids']
9. FranchisePlayers - requires: ['team_id']
10. InfographicFanDuelPlayer - requires: ['game_id']
... and 54 more


In [3]:
import json
from collections import defaultdict, Counter
import pandas as pd

# Load the full endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    endpoint_config = json.load(f)

def categorize_endpoints_by_params():
    """Categorize all endpoints by their required parameters"""
    
    # Group endpoints by required parameters pattern
    param_groups = defaultdict(list)
    param_combinations = Counter()
    
    # Collect all unique parameters
    all_params = set()
    
    for endpoint_name, endpoint_info in endpoint_config['endpoints'].items():
        required_params = endpoint_info['required_params']
        all_params.update(required_params)
        
        # Create a sorted tuple for consistent grouping
        param_key = tuple(sorted(required_params))
        param_groups[param_key].append(endpoint_name)
        param_combinations[param_key] += 1
    
    return param_groups, param_combinations, all_params

# Run the analysis
param_groups, param_combinations, all_params = categorize_endpoints_by_params()



In [4]:
# Import all endpoints at module level
from nba_api.stats import endpoints

# Create a generator for testing endpoints one by one
def endpoint_generator():
    """Generator that yields one endpoint at a time for testing"""
    for param_tuple, endpoint_list in param_groups.items():
        for endpoint_name in endpoint_list:
            yield param_tuple, endpoint_name

# Initialize the generator
endpoint_iter = endpoint_generator()

def test_next_endpoint():
    """Test the next endpoint and return the dataframe"""
    try:
        param_tuple, endpoint_name = next(endpoint_iter)
        
        print(f"Testing: {endpoint_name}")
        print(f"Required params: {', '.join(param_tuple) if param_tuple else 'None'}")
        
        # Get the endpoint class using getattr
        endpoint_class = getattr(endpoints, endpoint_name)
        
        # Build parameters
        kwargs = {}
        missing_params = []
        
        for param in param_tuple:
            if param in SAMPLE_PARAMS:
                kwargs[param] = SAMPLE_PARAMS[param]
            else:
                missing_params.append(param)
        
        if missing_params:
            print(f"Skipping - need values for: {missing_params}")
            return None
        
        # Create instance and get data
        instance = endpoint_class(**kwargs)
        dataframes = instance.get_data_frames()
        
        if dataframes:
            df = dataframes[0]
            print(f"Shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            return df
        else:
            print("No dataframes returned")
            return None
            
    except StopIteration:
        print("All endpoints tested!")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None



In [5]:
# Add 'usable' field to all endpoints and update JSON
def add_usable_field_to_endpoints():
    """Add 'usable' field to all endpoints (set to null initially)"""
    updated = False
    for endpoint_name, endpoint_info in endpoint_config['endpoints'].items():
        if 'usable' not in endpoint_info:
            endpoint_info['usable'] = None  # null/nan equivalent
            updated = True
    
    if updated:
        # Save updated config back to file
        with open('endpoint_priority_review.json', 'w') as f:
            json.dump(endpoint_config, f, indent=2)
        print("Added 'usable' field to all endpoints and saved to file")
    else:
        print("'usable' field already exists for all endpoints")

# Update categorization to filter out already-reviewed endpoints
def categorize_unreviewed_endpoints():
    """Categorize only endpoints that haven't been reviewed for usability yet"""
    
    # Group endpoints by required parameters pattern (only unreviewed ones)
    param_groups_unreviewed = defaultdict(list)
    param_combinations_unreviewed = Counter()
    
    total_endpoints = 0
    unreviewed_endpoints = 0
    
    for endpoint_name, endpoint_info in endpoint_config['endpoints'].items():
        total_endpoints += 1
        
        # Only include endpoints where usable is null/None (not yet reviewed)
        if endpoint_info.get('usable') is None:
            unreviewed_endpoints += 1
            required_params = endpoint_info['required_params']
            
            # Create a sorted tuple for consistent grouping
            param_key = tuple(sorted(required_params))
            param_groups_unreviewed[param_key].append(endpoint_name)
            param_combinations_unreviewed[param_key] += 1
    
    print(f"Total endpoints: {total_endpoints}")
    print(f"Unreviewed endpoints: {unreviewed_endpoints}")
    print(f"Already reviewed: {total_endpoints - unreviewed_endpoints}")
    
    return param_groups_unreviewed, param_combinations_unreviewed

# Run the setup
add_usable_field_to_endpoints()
param_groups_unreviewed, param_combinations_unreviewed = categorize_unreviewed_endpoints()

'usable' field already exists for all endpoints
Total endpoints: 136
Unreviewed endpoints: 2
Already reviewed: 134


In [6]:
# Updated generator for unreviewed endpoints only
def unreviewed_endpoint_generator():
    """Generator that yields only unreviewed endpoints for testing"""
    for param_tuple, endpoint_list in param_groups_unreviewed.items():
        for endpoint_name in endpoint_list:
            yield param_tuple, endpoint_name

# Initialize the unreviewed generator
endpoint_iter = unreviewed_endpoint_generator()

# Track current endpoint being reviewed
current_endpoint = None

def test_next_endpoint():
    """Test the next unreviewed endpoint and return the dataframe"""
    global current_endpoint
    
    try:
        param_tuple, endpoint_name = next(endpoint_iter)
        current_endpoint = endpoint_name  # Track current endpoint
        
        print(f"Testing: {endpoint_name}")
        print(f"Required params: {', '.join(param_tuple) if param_tuple else 'None'}")
        
        # Get the endpoint class using getattr
        endpoint_class = getattr(endpoints, endpoint_name)
        
        # Build parameters
        kwargs = {}
        missing_params = []
        
        for param in param_tuple:
            if param in SAMPLE_PARAMS:
                kwargs[param] = SAMPLE_PARAMS[param]
            else:
                missing_params.append(param)
        
        if missing_params:
            print(f"Skipping - need values for: {missing_params}")
            return None
        
        # Create instance and get data
        instance = endpoint_class(**kwargs)
        dataframes = instance.get_data_frames()
        
        if dataframes:
            df = dataframes[0]
            print(f"Shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            return df
        else:
            print("No dataframes returned")
            return None
            
    except StopIteration:
        print("All unreviewed endpoints tested!")
        current_endpoint = None
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def mark_usable(usable_value):
    """Mark the current endpoint as usable (True) or not usable (False)"""
    global current_endpoint
    
    if current_endpoint is None:
        print("No endpoint currently being reviewed!")
        return
    
    if usable_value not in [True, False]:
        print("usable_value must be True or False")
        return
    
    # Update the endpoint config
    endpoint_config['endpoints'][current_endpoint]['usable'] = usable_value
    
    # Save to file
    with open('endpoint_priority_review.json', 'w') as f:
        json.dump(endpoint_config, f, indent=2)
    
    print(f"✅ Marked {current_endpoint} as usable = {usable_value}")
    print("Updated and saved to endpoint_priority_review.json")

# Quick helper functions
def mark_true():
    """Quick function to mark current endpoint as usable = True"""
    mark_usable(True)

def mark_false():
    """Quick function to mark current endpoint as usable = False"""
    mark_usable(False)

print("Functions ready:")
print("- test_next_endpoint() : Test next unreviewed endpoint")
print("- mark_true()          : Mark current endpoint as usable = True") 
print("- mark_false()         : Mark current endpoint as usable = False")
print("- mark_usable(True/False) : Mark current endpoint with specific value")

Functions ready:
- test_next_endpoint() : Test next unreviewed endpoint
- mark_true()          : Mark current endpoint as usable = True
- mark_false()         : Mark current endpoint as usable = False
- mark_usable(True/False) : Mark current endpoint with specific value


In [ ]:
# Test the first endpoint
#mark_true()  # Example of marking the endpoint as usable
#mark_false()
# df = test_next_endpoint()
# if df is not None:
#     display(df.head())

✅ Marked FranchisePlayers as usable = False
Updated and saved to endpoint_priority_review.json
All unreviewed endpoints tested!


In [65]:
# CommonAllPlayers
# CommonPlayoffSeries
# DraftCombinePlayerAnthro
# LeagueDashLineups
# LeagueDashOppPtShot
# LeagueDashPlayerBioStats
# LeagueDashPlayerShotLocations
# LeagueDashPtDefend
# LeagueDashPtStats
# LeagueDashPtTeamDefend
# LeagueDashTeamPtShot
# LeagueDashTeamShotLocations
# LeagueDashTeamStats
# LeagueGameFinder
# LeagueGameLog
# LeagueHustleStatsPlayer
# LeagueHustleStatsTeam
# LeagueSeasonMatchups
# PlayerGameLogs
from nba_api.stats.endpoints import CommonAllPlayers
CommonAllPlayers(league_id='00').get_data_frames()[0] 


,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,,,,,None,Y,00
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,,,,,None,Y,00
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,,,,,None,Y,00
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,,,,,None,Y,00
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,,,,,None,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,78650,"Zunic, Matt",Matt Zunic,0,1948,1948,HISTADD_matt_zunic,matt_zunic,0,,,,,None,Y,00
5118,1641783,"da Silva, Tristan",Tristan da Silva,1,2024,2025,tristan_da_silva,tristan_da_silva,1610612753,Orlando,Magic,ORL,magic,magic,Y,00
5119,1628427,"Čančar, Vlatko",Vlatko Čančar,1,2019,2025,vlatko_cancar,vlatko_čančar,1610612743,Denver,Nuggets,DEN,nuggets,nuggets,Y,01
5120,1642365,"Đurišić, Nikola",Nikola Đurišić,0,2025,2025,nikola_durisic,nikola_đurišić,0,,,,,None,N,11


In [58]:
# Analysis: Count endpoints that are both usable and latest version
import json
from collections import defaultdict

# Load the current endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    config = json.load(f)

# Count endpoints by different criteria
total_endpoints = len(config['endpoints'])
usable_true = 0
usable_false = 0
usable_none = 0
latest_version_true = 0
latest_version_false = 0
both_usable_and_latest = 0

# Analyze each endpoint and group by required params
usable_and_latest_endpoints = []
usable_and_latest_by_params = defaultdict(list)

for endpoint_name, endpoint_info in config['endpoints'].items():
    # Count usable status
    if endpoint_info.get('usable') is True:
        usable_true += 1
    elif endpoint_info.get('usable') is False:
        usable_false += 1
    else:
        usable_none += 1
    
    # Count latest_version status
    if endpoint_info.get('latest_version') is True:
        latest_version_true += 1
    else:
        latest_version_false += 1
    
    # Count both usable=True AND latest_version=True
    if endpoint_info.get('usable') is True and endpoint_info.get('latest_version') is True:
        both_usable_and_latest += 1
        usable_and_latest_endpoints.append(endpoint_name)
        
        # Group by required parameters
        required_params = endpoint_info.get('required_params', [])
        param_key = tuple(sorted(required_params)) if required_params else ()
        usable_and_latest_by_params[param_key].append(endpoint_name)

print("=== ENDPOINT ANALYSIS ===")
print(f"Total endpoints: {total_endpoints}")
print()
print("USABLE STATUS:")
print(f"  usable = True:  {usable_true}")
print(f"  usable = False: {usable_false}")
print(f"  usable = None:  {usable_none}")
print()
print("LATEST VERSION STATUS:")
print(f"  latest_version = True:  {latest_version_true}")
print(f"  latest_version = False: {latest_version_false}")
print()
print("🎯 BOTH USABLE AND LATEST VERSION:")
print(f"  Count: {both_usable_and_latest}")
print(f"  Percentage: {both_usable_and_latest/total_endpoints*100:.1f}%")

print(f"\n📊 USABLE & LATEST ENDPOINTS GROUPED BY REQUIRED PARAMETERS:")
print(f"Found {len(usable_and_latest_by_params)} different parameter combinations")

# Sort groups by number of endpoints (most common first)
sorted_param_groups = sorted(usable_and_latest_by_params.items(), key=lambda x: len(x[1]), reverse=True)

for param_tuple, endpoints in sorted_param_groups:
    if not param_tuple:
        param_str = "NO PARAMETERS"
    else:
        param_str = ", ".join(param_tuple)
    
    print(f"\n📋 REQUIRES: [{param_str}]")
    print(f"   Count: {len(endpoints)} endpoints")
    
    # Show all endpoints in this group (since these are our prime endpoints)
    for i, endpoint in enumerate(sorted(endpoints)):
        print(f"   {i+1:2d}. {endpoint}")

print(f"\n🎯 SUMMARY: {both_usable_and_latest} endpoints ready for data collection!")

=== ENDPOINT ANALYSIS ===
Total endpoints: 136

USABLE STATUS:
  usable = True:  63
  usable = False: 71
  usable = None:  2

LATEST VERSION STATUS:
  latest_version = True:  125
  latest_version = False: 11

🎯 BOTH USABLE AND LATEST VERSION:
  Count: 55
  Percentage: 40.4%

📊 USABLE & LATEST ENDPOINTS GROUPED BY REQUIRED PARAMETERS:
Found 5 different parameter combinations

📋 REQUIRES: [NO PARAMETERS]
   Count: 19 endpoints
    1. CommonAllPlayers
    2. CommonPlayoffSeries
    3. DraftCombinePlayerAnthro
    4. LeagueDashLineups
    5. LeagueDashOppPtShot
    6. LeagueDashPlayerBioStats
    7. LeagueDashPlayerShotLocations
    8. LeagueDashPtDefend
    9. LeagueDashPtStats
   10. LeagueDashPtTeamDefend
   11. LeagueDashTeamPtShot
   12. LeagueDashTeamShotLocations
   13. LeagueDashTeamStats
   14. LeagueGameFinder
   15. LeagueGameLog
   16. LeagueHustleStatsPlayer
   17. LeagueHustleStatsTeam
   18. LeagueSeasonMatchups
   19. PlayerGameLogs

📋 REQUIRES: [game_id]
   Count: 14 endpo

In [429]:
# Update policy for all BoxScore endpoints
import json

# Load the current endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    config = json.load(f)

# Find and update all BoxScore endpoints
boxscore_endpoints = []
updated_count = 0

for endpoint_name, endpoint_info in config['endpoints'].items():
    if 'BoxScore' in endpoint_name:
        boxscore_endpoints.append(endpoint_name)
        # Update policy to 'standard'
        if endpoint_info.get('policy') != 'standard':
            endpoint_info['policy'] = 'standard'
            updated_count += 1

print(f"Found {len(boxscore_endpoints)} BoxScore endpoints:")
for i, endpoint in enumerate(sorted(boxscore_endpoints)):
    print(f"  {i+1:2d}. {endpoint}")

print(f"\nUpdated policy to 'standard' for {updated_count} BoxScore endpoints")

# Save the updated configuration back to the file
with open('endpoint_priority_review.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Changes saved to endpoint_priority_review.json")

Found 21 BoxScore endpoints:
   1. BoxScoreAdvancedV2
   2. BoxScoreAdvancedV3
   3. BoxScoreDefensiveV2
   4. BoxScoreFourFactorsV2
   5. BoxScoreFourFactorsV3
   6. BoxScoreHustleV2
   7. BoxScoreMatchupsV3
   8. BoxScoreMiscV2
   9. BoxScoreMiscV3
  10. BoxScorePlayerTrackV2
  11. BoxScorePlayerTrackV3
  12. BoxScoreScoringV2
  13. BoxScoreScoringV3
  14. BoxScoreSimilarityScore
  15. BoxScoreSummaryV2
  16. BoxScoreTraditionalV2
  17. BoxScoreTraditionalV3
  18. BoxScoreUsageV2
  19. BoxScoreUsageV3
  20. GLAlumBoxScoreSimilarityScore
  21. HustleStatsBoxScore

Updated policy to 'standard' for 21 BoxScore endpoints
✅ Changes saved to endpoint_priority_review.json
